# 5. Texte analysieren 

## Inhalte 
1. Worthäufigkeiten zählen
2. Konkordanzen

In [6]:
import pathlib

import nltk
import pandas
from reader import fairytale_corpus_reader

DATA_DIR = pathlib.Path().cwd().parent.joinpath("data")
PROC_DATA_DIR = DATA_DIR.joinpath("03_processed")

## 1. Worthäufigkeiten zählen

Zunächst erstellen wir eine Liste mit von Strings, deren Häufigkeiten wir bestimmen möchten.
Eine liste kann mit der Funkiton `sorted()` sortiert werden (in diesem Fall alphabetisch).

Zu beachten ist hierbei, dass nur die exakte Form des Strings gesucht wird: Also bei Eingabe von "pferd" werden Formen wie "des Pferds", "die Pferde" oder "die Pferden" nicht berücksichtigt.
Es hierbei hierbei nur die Häufigkeiten des Singulars gezählt.

In [2]:
animals = sorted(["wolf", "pferd", "esel", "hund", "katze"])

In [3]:
# generate list to keep all counts
all_counts = []

# in this for loop, we iterate over each fairytale in the corpus
for fairytale in fairytale_corpus_reader.fileids():
    # we assign the fairytales words (lowercased) to a list, using a list comprehension
    tokens = [token.lower() for token in fairytale_corpus_reader.words(fairytale)]
    # we create a nltk.Text.object
    text = nltk.Text(tokens)
    vocab = text.vocab()
    # we create a list of each animal's count by using the text's vocabulary, which contains each token's count. Defaults to 0, if token is not in text.
    counts = [vocab[animal] for animal in animals]
    # we add the counts for a single fairytale to the list all_counts
    all_counts.append(counts)

# the list all_counts contains a list of counts for each animal in each fairytale
# the 2-dimensional list has number of fairytales x number of animals counts
print(all_counts[:5])

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 5, 0], [0, 0, 0, 0, 0], [0, 0, 0, 1, 3]]


In [7]:
# we create a pandas.DataFrame from the list of counts
# the rows in this 2-d matrix/table correspond to the fairytales
# the columns correspond to the animals
# a value of zero means the animal (singular form) is not mentioned in this text
fairytales_animals_counts = pandas.DataFrame(all_counts, columns=animals)
fairytales_animals_counts.insert(0, "title", fairytale_corpus_reader.fileids())
fairytales_animals_counts.head()

,title,esel,hund,katze,pferd,wolf
0,grimm_Das_kluge_Grethel_395-397.txt,0,0,0,0,0
1,grimm_allerleirauh_353-359.txt,0,0,0,0,0
2,grimm_aschenputtel_119-126.txt,0,0,0,5,0
3,grimm_bruder_lustig_402-413.txt,0,0,0,0,0
4,grimm_bruederchen_und_schwesterchen__057-064.txt,0,0,0,1,3


In [8]:
# sum up counts per animal over all fairytales, using pandas
# using .sum() we can sum the values per column, the result is a 1-dimensional list / pandas.Series containing only the counts
# using .sort_values() we can sort these values
fairytales_animals_counts[animals].sum().sort_values(ascending=False)

pferd    138
wolf     127
katze     82
hund      66
esel      51
dtype: int64

In [9]:
# getting descriptive statistics over counts, using pandas
fairytales_animals_counts.describe()

,esel,hund,katze,pferd,wolf
count,209.000000,209.000000,209.000000,209.000000,209.000000
mean,0.244019,0.315789,0.392344,0.660287,0.607656
std,1.519987,1.405959,2.127914,1.981744,2.812940
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000
max,16.000000,13.000000,22.000000,16.000000,18.000000


In [10]:
# select the fairytale (row) in the table, that has the highest count for the animal using a variable "query"
query = "wolf"

max_count_id = fairytales_animals_counts[query].argmax()
max_count_title = fairytales_animals_counts.loc[max_count_id].title
fairytales_animals_counts.loc[max_count_id]

title    grimm_der_wolf_und_der_fuchs_383-385.txt
esel                                            0
hund                                            0
katze                                           0
pferd                                           0
wolf                                           18
Name: 87, dtype: object

In [11]:
# how long is this fairytale?
len(fairytale_corpus_reader.words(max_count_title))

799

In [12]:
# store counts table on disk
filename = PROC_DATA_DIR.joinpath("fairytales-animals-counts.tsv")
fairytales_animals_counts.to_csv(filename, sep="\t", index=False)

## 2. Konkordanzen

Das NLTK Text-Objekt ermöglicht die Ausgabe von Konkordanzen zu einem Suchbegriff.

Verwendung: `text.concordance("suchbegriff")` oder `text.concordance(["suchbegriff1", "suchbegriff2"])`

Die Ausgabe ist eine Darstellung des Suchbegriffs im Kontext.

Für die Weiterverarbeiung der Ergebnisse jenseits einer Darstellung, gibt es die Methode `.concordancelist()`, die eine Liste zurückgibt.

Beide Methoden basieren auf dem `ConcordanceIndex`. Wenn es um mehr als eine explorative Analyse von Texten geht, sollte direkt mit dem `ConcordanceIndex` gearbeitet werden.

Siehe: 
- https://www.nltk.org/howto/concordance.html
- https://www.nltk.org/api/nltk.text.html#nltk.text.ConcordanceIndex

In [13]:
# We create a nltk.Text object, that contains all fairytales
tokens = [token.lower() for token in fairytale_corpus_reader.words()]
grimm_text = nltk.Text(tokens)

In [14]:
?grimm_text.concordance

Signature: grimm_text.concordance(word, width=79, lines=25)
Docstring:
Prints a concordance for ``word`` with the specified context window.
Word matching is not case-sensitive.

:param word: The target word or phrase (a list of strings)
:type word: str or list
:param width: The width of each line, in characters (default=80)
:type width: int
:param lines: The number of lines to display (default=25)
:type lines: int

:seealso: ``ConcordanceIndex``
File:      c:\ubexport\python\poetrycache\virtualenvs\textarbeit-mit-python-hhaa_esp-py3.10\lib\site-packages\nltk\text.py
Type:      method

In [15]:
# the method .concordance() displays the surrounding text of a search string or list of strings.

query = "wolf"
grimm_text.concordance(query, lines=5)

Displaying 5 of 127 matches:
rach „ wer aus mir trinkt , wird ein wolf : wer aus mir trinkt , wird ein wolf
wolf : wer aus mir trinkt , wird ein wolf .“ da rief das schwesterchen „ brüde
h , trink nicht , sonst wirst du ein wolf und frissest mich .“ das brüderchen 
n , und zwei tauben zerrupften einen wolf , zwei kinder die wurfen zwei zickle
am ein neues unglück . ein hungriger wolf lief heran und verschlang den ganzen


In [16]:
query = ["es", "war", "einmal"]
grimm_text.concordance(query, lines=5)

Displaying 5 of 66 matches:
eimbrächte . 65 . allerleirauh . es war einmal ein könig , der hatte eine frau 
 gestraft . 81 . bruder lustig . es war einmal ein großer krieg , und als der k
r ende . 116 . das blaue licht . es war einmal ein soldat , der hatte dem könig
 . 117 . das eigensinnige kind . es war einmal ein kind eigensinnig und that ni
ann .“ 152 . das hirtenbüblein . es war einmal ein hirtenbübchen , das war wege


In [17]:
# using a for loop over the list of animals, we display the first 10 concordances of each animal
# Keep in mind "der esel, die esel" is different from "der wolf, die woelfe".

for animal in animals:
    grimm_text.concordance(animal, lines=10)

Displaying 10 of 51 matches:
ieber gar kein kind gehabt als einen esel , und sagte man sollt ihn ins wasser
ine brücke . da sah ich einen jungen esel mit einer silbernen nase , der jagte
er sah mit schrecken daß er in einen esel verwandelt war . doch weil er dabei 
zur belohnung zwei mit gold beladene esel , die mußten ihm nachfolgen . zuletz
ihm ebenfalls zwei mit gold beladene esel . endlich langte das glückskind dahe
es teufels , und als dieser die vier esel mit dem golde sah , ward er ganz ver
 vierfüßige gethier berufen , ochs , esel , rind , hirsch , reh , und was die 
musikanten . es hatte ein mann einen esel , der schon lange jahre die säcke un
us dem futter zu schaffen , aber der esel merkte daß kein guter wind wehte , l
 jappst du so , packan ?“ fragte der esel . „ ach ,“ sagte der hund , „ weil i
Displaying 10 of 66 matches:
s könnte ja auch eine katze oder ein hund sein .“ da ließ sich der mann überre
ssinnen up un segden „ de verfluchte hund , usse bloet soll örfer die rac

In [18]:
import session_info

session_info.show()